# SDC

### **import**

In [1]:
import pandas as pd
import pyodbc
import sqlite3
from datetime import datetime

C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_20288\2168663434.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### **Database connection**

In [2]:
DB = {'servername': 'LAPTOP-LPE28RPE\SQLEXPRESS', 
    'database': 'DEDS'}

export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + 
                              ';DATABASE=' + DB['database'])

export_cursor = export_conn.cursor()
export_cursor   

go_sales_con = sqlite3.connect('Great_Outdoors_Data_SQLite\go_sales.sqlite')
go_crm_con = sqlite3.connect('Great_Outdoors_Data_SQLite\go_crm.sqlite')
go_staff_con = sqlite3.connect('Great_Outdoors_Data_SQLite\go_staff.sqlite')
go_inv_con = 'Great_Outdoors_Data_SQLite\GO_SALES_INVENTORY_LEVELSData.csv'
go_forecast_con = 'Great_Outdoors_Data_SQLite\GO_SALES_PRODUCT_FORECASTData.csv'

### **Dataframes aanmaken**

In [3]:
Go_staff_queries = {
    'Course': 'SELECT * FROM Course',
    'Training': 'SELECT * FROM Training',
    'Sales_staff': 'SELECT * FROM Sales_staff',
    'Satisfaction': 'SELECT * FROM Satisfaction',
    'Satisfaction_type': 'SELECT * FROM Satisfaction_type',
}

Go_crm_queries = {
    'Retailer_contact': 'SELECT * FROM Retailer_contact',
    'Retailer_site' : 'SELECT * FROM Retailer_site',
    'Retailer' : 'SELECT * FROM Retailer',
    'Retailer_type' : 'SELECT * FROM Retailer_type',
    'COUNTRY': 'SELECT * FROM COUNTRY',
    'Sales_territory': 'SELECT * FROM Sales_territory'
}

Go_sales_queries = {
    'Order_method': 'SELECT * FROM Order_method',
    'Order_header': 'SELECT * FROM Order_header',
    'Order_details': 'SELECT * FROM Order_details',
    'Returned_item': 'SELECT * FROM Returned_item',
    'Return_reason': 'SELECT * FROM Return_reason',
    'Product': 'SELECT * FROM Product',
    'PRODUCT_TYPE': 'SELECT * FROM PRODUCT_TYPE',
    'PRODUCT_LINE' : 'SELECT * FROM PRODUCT_LINE',
    'Sales_TARGETData' : 'SELECT * FROM Sales_TARGETData',
    'Sales_branch': 'SELECT * FROM Sales_branch'
}


dataframes = {}

# Lees elke tabel in een DataFrame
for table_name, query in Go_staff_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, go_staff_con)

for table_name, query in Go_crm_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, go_crm_con)

for table_name, query in Go_sales_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, go_sales_con)

# Lees de CSV-bestanden in pandas DataFrames
GO_SALES_INVENTORY_LEVELSData = pd.read_csv(go_inv_con, index_col=False)
GO_SALES_PRODUCT_FORECASTData = pd.read_csv(go_forecast_con, index_col=False)

# Voeg de DataFrames toe aan de dataframes dictionary
dataframes['GO_SALES_INVENTORY_LEVELSData'] = GO_SALES_INVENTORY_LEVELSData
dataframes['GO_SALES_PRODUCT_FORECASTData'] = GO_SALES_PRODUCT_FORECASTData

#als je ik elk tabel als een dataframe/ variabele wil behandelen of aanroepen moet ik dit uitvoeren.
for table_name, df in dataframes.items():
    globals()[table_name] = df

Sales_staff['DATE_HIRED'] = pd.to_datetime(Sales_staff['DATE_HIRED'], errors='coerce')
Sales_staff['DATE_HIRED'] = Sales_staff['DATE_HIRED'].dt.date

C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_20288\4046988446.py:56: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  Sales_staff['DATE_HIRED'] = pd.to_datetime(Sales_staff['DATE_HIRED'], errors='coerce')


### **Course_dimensie loaden**

In [4]:
course_dimensie = Course.rename(columns={
    'COURSE_CODE': 'Course_COURSE_CODE',
    'COURSE_DESCRIPTION': 'Course_COURSE_DESCRIPTION'
})


for index, row in course_dimensie.iterrows():

    try:
        query= """ INSERT INTO Course (Course_COURSE_CODE, Course_COURSE_DESCRIPTION) VALUES (?, ?)"""
        params = (
            row['Course_COURSE_CODE'], 
            row['Course_COURSE_DESCRIPTION']
        )
        export_cursor.execute(query, params)
    except pyodbc.Error as e:
        print(f"An error occurred: {e}")
        print(query)

export_conn.commit()



### **Training_feit loaden**

In [5]:
merge1 = pd.merge(Training, Sales_staff, on = 'SALES_STAFF_CODE')

Training_feit = pd.merge(merge1, Course, on = 'COURSE_CODE')


Training_feit = Training_feit.rename(columns = {
    'YEAR' : 'Year_nr',
    'SALES_STAFF_CODE' : 'Training_Sales_staff_code',
    'COURSE_CODE' : 'Course_COURSE_CODE'
})

for index, row in Training_feit.iterrows():
    # Voor elke Training rij, vind de meest recente Course_SK gebaseerd op Course_CODE
    query = """
    SELECT TOP 1 Course_SK
    FROM Course
    WHERE Course_COURSE_CODE = ?
    ORDER BY [Time-stamp] DESC  -- Selecteert op basis van de meest recente Timestamp
    """
    export_cursor.execute(query, (row['Course_COURSE_CODE'],))
    result = export_cursor.fetchone()

    if result:
        course_sk = result[0]
        # Voeg de Course_SK in bij de bijbehorende Training rij met de juiste Course_FSK
        insert_query = """
        INSERT INTO Training (Year_nr, Training_Sales_staff_code, Course_COURSE_CODE, Course_FSK) 
        VALUES (?, ?, ?, ?)
        """
        params = (
            row['Year_nr'],
            row['Training_Sales_staff_code'],
            row['Course_COURSE_CODE'],
            course_sk  # De gevonden Course_SK
        )
        try:
            export_cursor.execute(insert_query, params)
        except pyodbc.Error as e:
            print(f"Een fout opgetreden: {e}")
    else:
        print(f"Geen overeenkomstige Course_SK gevonden voor Course_COURSE_CODE {row['Course_COURSE_CODE']}")

export_conn.commit()



### **Data_mutatie simuleren**

**Course**

In [6]:
#Data om toe te voegen
Nieuwe_data = pd.DataFrame([['10', 'GO New Course']], columns=['COURSE_CODE', 'COURSE_DESCRIPTION'])

# Voeg nieuwe rij toe aan het DataFrame
Course = pd.concat([Course, Nieuwe_data], ignore_index=True)

# Update bestaande data
Course.loc[Course['COURSE_CODE'] == '2', 'COURSE_DESCRIPTION'] = 'GO Updated Course'
#Course.loc[Course['COURSE_CODE'] == '2', 'COURSE_DESCRIPTION'] = 'GO Updated Course opnieuw'

**Training**

In [11]:
# alle kolommen zijn pk's dus niks om te updaten
Nieuwe_data = pd.DataFrame([['2008', '99', '2']], columns=['Year_nr', 'Training_Sales_staff_code', 'Course_COURSE_CODE'])
Training_feit = pd.concat([Training_feit, Nieuwe_data], ignore_index=True)

### **Controleren en toevoegen**

**Course logica**

In [9]:
course_dimensie = Course.rename(columns={
    'COURSE_CODE': 'Course_COURSE_CODE',
    'COURSE_DESCRIPTION': 'Course_COURSE_DESCRIPTION'
})


for index, row in course_dimensie.iterrows():
    # Controleer of de rij bestaat en haal de huidige beschrijving op
    export_cursor.execute("SELECT Course_COURSE_DESCRIPTION FROM Course WHERE Course_COURSE_CODE = ?", row['Course_COURSE_CODE'])
    result = export_cursor.fetchone()

    if result is None:
        # Rij bestaat niet, voeg toe
        query = """INSERT INTO Course (Course_COURSE_CODE, Course_COURSE_DESCRIPTION) VALUES (?, ?)"""
        params = (row['Course_COURSE_CODE'], row['Course_COURSE_DESCRIPTION'])
    elif result[0] != row['Course_COURSE_DESCRIPTION']:
        # Rij bestaat en beschrijving is anders, beschouw dit als een "update" en voeg een nieuwe rij toe
        query = """INSERT INTO Course (Course_COURSE_CODE, Course_COURSE_DESCRIPTION) VALUES (?, ?)"""
        params = (row['Course_COURSE_CODE'], row['Course_COURSE_DESCRIPTION'])
    else:
        # Rij bestaat en beschrijving is hetzelfde, doe niets
        continue  # Ga verder met de volgende iteratie van de loop

    try:
        export_cursor.execute(query, params)
    except pyodbc.Error as e:
        print(f"An error occurred: {e}")
        print(query)

export_conn.commit()

**Training logica**

In [13]:
for index, row in Training_feit.iterrows():
    # Controleer eerst of de rij al bestaat in de Training tabel
    check_query = """
    SELECT COUNT(*)
    FROM Training
    WHERE Year_nr = ? AND Training_Sales_staff_code = ? AND Course_COURSE_CODE = ?
    """
    export_cursor.execute(check_query, (row['Year_nr'], row['Training_Sales_staff_code'], row['Course_COURSE_CODE']))
    count_result = export_cursor.fetchone()

    if count_result[0] == 0:
        # Rij bestaat niet, zoek naar de meest recente Course_SK voor de gegeven Course_COURSE_CODE
        get_course_sk_query = """
        SELECT TOP 1 Course_SK
        FROM Course
        WHERE Course_COURSE_CODE = ?
        ORDER BY [Time-stamp] DESC
        """
        export_cursor.execute(get_course_sk_query, (row['Course_COURSE_CODE'],))
        course_sk_result = export_cursor.fetchone()

        if course_sk_result:
            course_sk = course_sk_result[0]
            # Voeg de nieuwe rij toe aan de Training tabel met de meest recente Course_FSK
            insert_query = """
            INSERT INTO Training (Year_nr, Training_Sales_staff_code, Course_COURSE_CODE, Course_FSK) 
            VALUES (?, ?, ?, ?)
            """
            params = (
                row['Year_nr'],
                row['Training_Sales_staff_code'],
                row['Course_COURSE_CODE'],
                course_sk  # De gevonden Course_SK
            )
            try:
                export_cursor.execute(insert_query, params)
            except pyodbc.Error as e:
                print(f"Een fout opgetreden: {e}")
        else:
            print(f"Geen overeenkomstige Course_SK gevonden voor Course_COURSE_CODE {row['Course_COURSE_CODE']}")
    #else:
       # print(f"Rij bestaat al voor Year_nr {row['Year_nr']}, Training_Sales_staff_code {row['Training_Sales_staff_code']}, Course_COURSE_CODE {row['Course_COURSE_CODE']}")

export_conn.commit()


